In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# kernel Python 3.11.11 /local/bin/python

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
! pwd
! pip install -r requirements.txt
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

/kaggle/working/kaggle/titanic


In [3]:
! pip install skorch>=0.12.0

In [4]:
import sys, os

repo_root = "/kaggle/working/kaggle"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
import warnings
warnings.filterwarnings(
    "ignore",
    message="Found unknown categories in columns [0, 2, 3] during transform. These unknown categories will be encoded as all zeros",
    category=UserWarning
)
from common.preprocessing import make_preprocessor
from common.pipelines     import compare_models, grid_search
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from titanic_features import add_features
import joblib
from xgboost import XGBClassifier  
from sklearn.metrics       import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint, uniform
import skorch

In [5]:
# load straight from Kaggle’s mount
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test  = pd.read_csv('/kaggle/input/titanic/test.csv')

In [8]:
# apply to train & capture bins
# train, fare_bins, age_fill, fare_fill, embarked_fill = add_features(train)

# # apply to test _using_ the same fare_bins
# test, _, _, _, _ = add_features(
#     test,
#     fare_bins=fare_bins,
#     age_fill=age_fill,
#     fare_fill=fare_fill,
#     embarked_fill=embarked_fill
# )

# NUM_COLS = ['Age','Fare','FamilySize']
# CAT_COLS = ['Pclass','Sex','Embarked',
#             'Title','IsAlone','Deck','FareBin','AgeBin']

NUM_COLS = ['Age','Fare']
CAT_COLS = ['Pclass','Sex','Embarked']

TARGET   = 'Survived'

# X = train[NUM_COLS + CAT_COLS]
# y = train[TARGET]
# X_test  = test[NUM_COLS  + CAT_COLS]

X = train[NUM_COLS + CAT_COLS]
y = train[TARGET]

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

preprocessor = make_preprocessor(NUM_COLS, CAT_COLS,cat_encoder="onehot")

# fit on train, transform both
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc   = preprocessor.transform(X_val)


In [12]:
X_train_proc.shape

(712, 8)

In [ ]:
import torch.nn as nn
from skorch import NeuralNetClassifier
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import FunctionTransformer

torch.set_default_dtype(torch.float64)
# 1) define your PyTorch module
class TitanicNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),      nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),        nn.Sigmoid(),
        )

    def forward(self, X):
        return self.net(X).squeeze(-1)

# 3) build pipeline
preprocessor = joblib.load("titanic_preprocessor.joblib")
n_features = preprocessor.transform(X_train_proc).shape[1]

print(n_features)

# 2) wrap in skorch
net = NeuralNetClassifier(
    module=TitanicNet,
    module__input_dim=n_features,
    max_epochs=30,
    lr=1e-3,
    batch_size=32,
    optimizer=torch.optim.Adam,
    criterion=nn.BCELoss,
    train_split=None,       # or use skorch callbacks for val
    verbose=1,
    device=device           # or 'cpu'
)
net.initialize() 
net.module_.double()
to_float64 = FunctionTransformer(lambda X: X.astype(np.float64))

pipeline = Pipeline([
    ("preproc", preprocessor),
    ("to64",       to_float64),
    ("nn",      net),
])

# 4) fit & evaluate
pipeline.fit(X_train_proc, y_train)
print("Train score:", pipeline.score(X_train, y_train))
print("Val   score:", pipeline.score(X_val_proc,   y_val))


ValueError: X has 8 features, but ColumnTransformer is expecting 11 features as input.

In [14]:
from sklearn.pipeline import Pipeline
import torch.nn as nn

class TitanicNetLazy(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.LazyLinear(256),   # ← will auto-infer in_features
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, X):
        return self.net(X).squeeze(-1)
    
# 1) build just the preprocessor + dtype cast
preproc_pipe = Pipeline([
    # ("preproc", preprocessor),           # your column transformer
    ("to64",   FunctionTransformer(lambda X: X.astype(np.float64))),
])

# 2) figure out exactly how many columns it produces
n_features = preproc_pipe.transform(X_train_proc).shape[1]

# 3) use *that* when constructing the net
net = NeuralNetClassifier(
    module=TitanicNetLazy,    # LazyLinear will infer the input size
    max_epochs=30,
    lr=1e-3,
    batch_size=32,
    optimizer=torch.optim.Adam,
    criterion=nn.BCELoss,
    train_split=None,
    verbose=1,
    device=device,
)


# 4) now build your full pipeline
pipeline = Pipeline([
    ("preproc_pipe", preproc_pipe),
    ("net",          net),
])
y_train_double = y_train.astype(np.float64)
y_val_double   = y_val.astype(np.float64)
pipeline.fit(X_train_proc, y_train_double)


  epoch    train_loss     dur
-------  ------------  ------
      1        0.6231  0.3155
      2        0.5469  0.0868
      3        0.5094  0.0837
      4        0.4846  0.0746
      5        0.4734  0.0658
      6        0.4563  0.0651
      7        0.4654  0.0636
      8        0.4532  0.0615
      9        0.4502  0.0615
     10        0.4391  0.0593
     11        0.4292  0.0610
     12        0.4330  0.0620
     13        0.4307  0.0601
     14        0.4325  0.0607
     15        0.4193  0.0593
     16        0.4345  0.0591
     17        0.4190  0.0576
     18        0.4087  0.0602
     19        0.4086  0.0589
     20        0.4244  0.0594
     21        0.4293  0.0609
     22        0.4142  0.0587
     23        0.4243  0.0585
     24        0.4124  0.0595
     25        0.4120  0.0577
     26        0.4060  0.0640
     27        0.4150  0.0583
     28        0.4321  0.0597
     29        0.4051  0.0598
     30        0.4023  0.0604


Pipeline(steps=[('preproc_pipe',
                 Pipeline(steps=[('to64',
                                  FunctionTransformer(func=<function <lambda> at 0x7cb4f08ad440>))])),
                ('net',
                 NeuralNetClassifier(_params_to_validate=set(), batch_size=32, callbacks=None, compile=False, criterion=<class 'torch.nn.modules.loss.BCELoss'>, dataset=<class 'skorch.dataset.Dataset'>, device='cuda', iterator_train=<class 'torch.utils.data.dataloader.DataLoader'>, iterator_valid=<class 'torch.utils.data.dataloader.DataLoader'>, lr=0.001, max_epochs=30, module=<class '__main__.TitanicNetLazy'>, optimizer=<class 'torch.optim.adam.Adam'>, predict_nonlinearity='auto', torch_load_kwargs=None, train_split=None, use_caching='auto', verbose=1, warm_start=False))])

In [19]:
print("Val score:", pipeline.score(X_val, y_val_double))

Val score: 0.8212290502793296


In [15]:
print("Val score:", pipeline.score(X_val_proc, y_val_double))

Val score: 0.8100558659217877
